In [1]:
import folium
import requests
import numpy as np
import pandas as pd
import geopy

In [2]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', flavor='html5lib')

In [3]:
df = df[0]
pc = df[0]
bors = df[1]
neigh = df[2]
pc.drop(0,inplace=True)
bors.drop(0,inplace=True)
neigh.drop(0,inplace=True)

#### Creating a new dataframe to store all columns

In [4]:
cols = ['PostalCode', 'Borough', 'Neighborhood']
df_codes = pd.DataFrame(columns=cols)

In [5]:
df_codes['PostalCode'] = pc
df_codes['Borough'] = bors
df_codes['Neighborhood'] = neigh

#### Removing the elements whose boroughs have not been assigned

In [6]:
df_codes.drop(df_codes[df_codes.Borough == "Not assigned"].index, inplace=True)

df_codes = df_codes.reset_index()

df_codes.drop(['index'], axis=1, inplace=True)

df_codes.head(10)


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


#### Copying the borough name to the neighborhood when it is not assigned

In [7]:
for i in range(len(df_codes)):
    if (df_codes['Neighborhood'][i] == "Not assigned"):
        df_codes['Neighborhood'][i] = df_codes['Borough'][i]
df_codes.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


#### Loop for grouping by postalcodes and merging same postalcode cells
##### Inds stores the indexes of the rows merged(which will be dropped later)

In [8]:
inds = []
for i in range(0,(len(df_codes)-1)):
    if( str(df_codes['PostalCode'].iloc[i]) == str(df_codes['PostalCode'].iloc[i+1])):
        df_codes['Neighborhood'].iloc[i] += ', '
        df_codes['Neighborhood'].iloc[i] += df_codes['Neighborhood'].iloc[i+1]
        inds.append(int(i+1))
df_codes.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,"Lawrence Heights, Lawrence Manor"
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,"Rouge, Malvern"
9,M1B,Scarborough,Malvern


In [9]:
df_codes.drop(inds, inplace=True)

In [10]:
df_codes = df_codes.reset_index()
df_codes.drop(['index'], axis=1, inplace=True)
df_codes.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [11]:
df_codes['Latitude'] = pd.Series(np.random.randn(len(df_codes)), index=df_codes.index)
df_codes['Longitude'] = pd.Series(np.random.randn(len(df_codes)), index=df_codes.index)

In [12]:
df2 = pd.read_csv('D:\DataSets\Geospatial_Coordinates.csv')
df2.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [13]:
df2[df2['Postal Code'] == 'M4A'].index.values.astype(int)[0]

34

In [14]:
for i in range(len(df_codes)):
    ind = df2[df2['Postal Code'] == str(df_codes['PostalCode'].iloc[i])].index.values.astype(int)[0]
    df_codes['Latitude'].iloc[i] = float(df2['Latitude'].iloc[ind])
    df_codes['Longitude'].iloc[i] = float(df2['Longitude'].iloc[ind])

D:\Anaconda\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [15]:
df_codes.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


In [16]:
TorLat = 43.6532
TorLon = -79.3832

In [17]:
mapTor = folium.Map(location=[TorLat, TorLon], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_codes['Latitude'], df_codes['Longitude'], df_codes['Borough'], df_codes['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(mapTor)  

    
mapTor

In [19]:
import json

In [20]:
CLIENT_ID = 'X2VYAOTLOCJK4X5CUB45HEUI2KD2HIBYKCMFU5FZTXSLLSDT' 
CLIENT_SECRET = 'QVFVSX2UNS0UN0MSDUM5TCS310EHHBI15Z2REYD2OLMSCHAR' 
VERSION = '20181230'

In [21]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [22]:
Torvenues = getNearbyVenues(names = df_codes['Neighborhood'],
                            latitudes = df_codes['Latitude'],
                            longitudes = df_codes['Longitude'])

Parkwoods
Victoria Village
Harbourfront, Regent Park
Lawrence Heights, Lawrence Manor
Queen's Park
Islington Avenue
Rouge, Malvern
Don Mills North
Woodbine Gardens, Parkview Hill
Ryerson, Garden District
Glencairn
Cloverdale, Islington
Highland Creek, Rouge Hill
Flemingdon Park, Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens, Eringate
Guildwood, Morningside
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Downsview North
Thorncliffe Park
Adelaide, King
Dovercourt Village, Dufferin
Scarborough Village
Fairview, Henry Farm
Northwood Park, York University
East Toronto
Harbourfront East, Toronto Islands
Little Portugal, Trinity
East Birchmount Park, Ionview
Bayview Village
CFB Toronto, Downsview East
The Danforth West, Riverdale
Design Exchange, Toronto Dominion Centre
Brockton, Exhibition Place
Clairlea, Golden Mile
Silver Hills, York Mills
Downsview West
The Beaches Wes

In [23]:
Torvenues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Parkwoods,43.753259,-79.329656,Bella Vita Catering & Private Chef Service,43.756651,-79.331524,BBQ Joint
4,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena


In [24]:
cuisine = input("")

cuisine = cuisine + " Restaurant"

Indian


In [25]:
df1 = Torvenues[Torvenues['Venue Category'] != cuisine]

In [26]:
df1 = pd.DataFrame(df1)
df1.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Parkwoods,43.753259,-79.329656,Bella Vita Catering & Private Chef Service,43.756651,-79.331524,BBQ Joint
4,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena


In [56]:
df2=pd.DataFrame(columns=[['Neighborhood','Neighborhood Latitude','Neighborhood Longitude', 'Venue','Venue Latitude','Venue Longitude','Venue Category']])

In [60]:
for i in range(len(df1)):
    if("Restaurant" in df1['Venue Category'][i]):
        pd.concat(df2,pd.DataFrame(df1.loc[i]).transpose())
    else:
        pass

In [80]:
df2.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
1,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
5,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
13,"Harbourfront, Regent Park",43.654260,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
22,"Harbourfront, Regent Park",43.654260,-79.360636,Cocina Economica,43.654959,-79.365657,Mexican Restaurant
29,"Harbourfront, Regent Park",43.654260,-79.360636,Archeo,43.650667,-79.359431,Italian Restaurant
